# Setting

In [ ]:
from urllib.parse import urlparse, urlunparse
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException 
import time, logging, os, re

In [2]:
# 기본 로거 생성
logging.basicConfig(level=logging.INFO, format='%(asctime)s | %(message)s')
logger = logging.getLogger()
logger.info('크롤링 시작')

2024-03-26 11:01:57,246 | 크롤링 시작


In [3]:
data_folder_path = os.path.join(os.getcwd(), "data")

try:
    os.mkdir(data_folder_path)
    print("data 폴더를 생성했습니다.")
except FileExistsError:
    print("data 폴더가 이미 존재합니다.")
except Exception as e:
    print("오류 발생:", e)

data 폴더가 이미 존재합니다.


In [4]:
# 수집할 리뷰 개수
n = 500

# 수집하고자 하는 상품의 URL
raw_url = 'https://smartstore.naver.com/yuri6721/products/4859018318?NaPm=ct%3Dlu6zf4z4%7Cci%3D5f4314623920d062e07b4b3484d399a7fc738f99%7Ctr%3Dslsl%7Csn%3D1198282%7Chk%3D6b2d7cf6cf2580807ebab0c5736b4bce70ac307d'

# URL 분해 -> schem, netloc, path, parms, query, fragement
parsed_url = urlparse(raw_url)

# parms, query, fragement 3가지 값을 공백으로 설정하여 URL 재구성
url = urlunparse((parsed_url.scheme, parsed_url.netloc, parsed_url.path + '/', '', '', ''))

logger.info(f'변경된 URL : {url}')

2024-03-26 11:01:57,261 | 변경된 URL : https://smartstore.naver.com/yuri6721/products/4859018318/


In [5]:
# 크롬 드라이버 설치
driver_path = ChromeDriverManager().install()
logger.info(f'크롬 드라이버 설치 경로 : {driver_path}')

# 브라우저 인스턴스 생성, url 오픈
driver = webdriver.Chrome(service=Service(driver_path))
driver.get(url)
driver.implicitly_wait(10)

2024-03-26 11:01:57,267 | ====== WebDriver manager ======
2024-03-26 11:01:57,815 | Get LATEST chromedriver version for google-chrome
2024-03-26 11:01:58,105 | Get LATEST chromedriver version for google-chrome
2024-03-26 11:01:58,356 | Get LATEST chromedriver version for google-chrome
2024-03-26 11:01:58,964 | WebDriver version 122.0.6261.128 selected
2024-03-26 11:01:58,966 | Modern chrome version https://storage.googleapis.com/chrome-for-testing-public/122.0.6261.128/win32/chromedriver-win32.zip
2024-03-26 11:01:58,966 | About to download new driver from https://storage.googleapis.com/chrome-for-testing-public/122.0.6261.128/win32/chromedriver-win32.zip
2024-03-26 11:01:59,285 | Driver downloading response is 200
2024-03-26 11:01:59,470 | Get LATEST chromedriver version for google-chrome
2024-03-26 11:01:59,805 | Driver has been saved in cache [C:\Users\njh26\.wdm\drivers\chromedriver\win64\122.0.6261.128]
2024-03-26 11:01:59,807 | 크롬 드라이버 설치 경로 : C:\Users\njh26\.wdm\drivers\chromedr

# Crawling

In [6]:
# 상품명 가져오기
product_name = driver.find_element(By.CLASS_NAME, '_22kNQuEXmb._copyable').text
product_name = product_name.replace(':', '')    # * 가 있는 경우 to_csv에서 오류 발생
### df.to_csv에서 오류가 발생한 경우 위 코드에서 문자를 추가로 변환해야 함
logger.info(f'상품명 : {product_name}')

2024-03-26 11:02:01,780 | 상품명 : 단양구경시장 유리네 수제 궁채장아찌 절임 1KG [원산지상세설명에 표시]


In [7]:
# 리뷰 탭으로 이동
review_tab = driver.find_element(By.XPATH, '//*[@id="content"]/div/div[3]/div[3]/ul/li[2]/a')
review_tab.click()

# 최신순 정렬로 변경
# 리뷰 탭으로 이동해야만 최신순 버튼을 찾을 수 있음
latest = driver.find_element(By.XPATH, '//*[@id="REVIEW"]/div/div[3]/div[1]/div[1]/ul/li[2]/a')
latest.click()
time.sleep(0.1)

In [8]:
# 리뷰를 저장할 리스트 생성
reviews = []
ratings = []
days = []
options = []
buyer_categorys = []
first_keywords = []
second_keywords = []
third_keywords = []

# 크롤링 함수
def crawling():

    # 리뷰 박스
    review_boxs = driver.find_elements(By.CSS_SELECTOR, 'ul._2ms2i3dD92 > li')

    # 각 리뷰 박스별로 데이터 수집
    for (j, review_box) in enumerate(review_boxs):

        # 리뷰
        review = review_box.find_element(By.CSS_SELECTOR,'div._1kMfD5ErZ6 > span._2L3vDiadT9')
        cleaned_text = re.sub(r'[\U0001F300-\U0001F6FF]', '', review.text)  # 이모지 제거 <- 파일 저장시 오류발생
        reviews.append(cleaned_text)

        # 별점
        rating = review_box.find_element(By.CSS_SELECTOR,'div._3HKlxxt8Ii > div._2V6vMO_iLm > em._15NU42F3kT')
        ratings.append(rating.text)
        
        # 리뷰작성일
        day = review_box.find_element(By.CSS_SELECTOR,'div.iWGqB6S4Lq > span._2L3vDiadT9')
        days.append('20' + day.text)  # 24.03.07 형식으로 수집되어 앞에 20 추가
        
        # 구매옵션
        # 키워드와 같이 있어 수집 이후에 \n 으로 슬라이싱
        option_and_keyword = review_box.find_element(By.CSS_SELECTOR,'div._3HKlxxt8Ii > div._2FXNMst_ak')
        options.append(option_and_keyword.text.split('\n')[0])

        # 구매자 정보
        try : 
            buyer_category = review_box.find_element(By.CSS_SELECTOR,'dl.XbGQRlzveO > div._3F8sJXhFeW')       
            buyer_categorys.append(buyer_category.text)
        except NoSuchElementException : 
            buyer_categorys.append('')  # 구매자 정보가 없는 경우 공백을 입력

        # 키워드
        keyword_all = review_box.find_elements(By.CSS_SELECTOR,'dl.XbGQRlzveO > div._1QLwBCINAr > dd')
        first_keywords.append(keyword_all[0].text)
        second_keywords.append(keyword_all[1].text)
        third_keywords.append(keyword_all[2].text)
        
        logger.info(f'\t {j+1}번째 리뷰 획득')

In [9]:
# 페이지 수
pages = int((n-1) / 20 + 1)   #페이지당 리뷰 20개

# 다음 페이지 버튼
next = driver.find_element(By.CLASS_NAME, 'fAUKm1ewwo._2Ar8-aEUTq')

# 크롤링
for i in range(1, pages + 1):
    if i == 1:
        crawling()
        logger.info(f'--{i} 페이지 리뷰 수집 완료--')
    else:
        # 다음 페이지로 이동
        next.click()
        time.sleep(0.5)
        crawling()
        logger.info(f'--{i} 페이지 리뷰 수집 완료--')

2024-03-26 11:02:13,713 | 	 1번째 리뷰 획득
2024-03-26 11:02:13,779 | 	 2번째 리뷰 획득
2024-03-26 11:02:23,871 | 	 3번째 리뷰 획득
2024-03-26 11:02:23,939 | 	 4번째 리뷰 획득
2024-03-26 11:02:34,040 | 	 5번째 리뷰 획득
2024-03-26 11:02:34,118 | 	 6번째 리뷰 획득
2024-03-26 11:02:44,193 | 	 7번째 리뷰 획득
2024-03-26 11:02:54,268 | 	 8번째 리뷰 획득
2024-03-26 11:03:04,335 | 	 9번째 리뷰 획득
2024-03-26 11:03:14,418 | 	 10번째 리뷰 획득
2024-03-26 11:03:14,482 | 	 11번째 리뷰 획득
2024-03-26 11:03:24,553 | 	 12번째 리뷰 획득
2024-03-26 11:03:24,617 | 	 13번째 리뷰 획득
2024-03-26 11:03:34,693 | 	 14번째 리뷰 획득
2024-03-26 11:03:44,779 | 	 15번째 리뷰 획득
2024-03-26 11:03:54,861 | 	 16번째 리뷰 획득
2024-03-26 11:04:04,918 | 	 17번째 리뷰 획득
2024-03-26 11:04:14,989 | 	 18번째 리뷰 획득
2024-03-26 11:04:15,054 | 	 19번째 리뷰 획득
2024-03-26 11:04:25,119 | 	 20번째 리뷰 획득
2024-03-26 11:04:25,120 | --1 페이지 리뷰 수집 완료--
2024-03-26 11:04:25,763 | 	 1번째 리뷰 획득
2024-03-26 11:04:35,820 | 	 2번째 리뷰 획득
2024-03-26 11:04:35,883 | 	 3번째 리뷰 획득
2024-03-26 11:04:35,947 | 	 4번째 리뷰 획득
2024-03-26 11:04:46,017 | 	 5번째 

In [10]:
# 키워드명 가져오기
keyword_name = driver.find_elements(By.CSS_SELECTOR,'dl.XbGQRlzveO > div._1QLwBCINAr > dt') 
keyword_name_list = [a.text for a in keyword_name][:3]

# 드라이버 종료
driver.quit()

In [11]:
# 크롤링 결과 저장
df = pd.DataFrame({
                'reviews': reviews, 
                'ratings': ratings, 
                'days': days,
                'options': options, 
                'buyer_categorys' : buyer_categorys,
                keyword_name_list[0] : first_keywords, 
                keyword_name_list[1] : second_keywords, 
                keyword_name_list[2] : third_keywords
                })

print(f'\n수집한 데이터는 다음과 같습니다. \n {df.head(4)}')

df.to_csv(f"data/{product_name}_{n}.csv", encoding='utf-8-sig')
logger.info('크롤링 완료')

2024-03-26 12:04:10,987 | 크롤링 완료



수집한 데이터는 다음과 같습니다. 
                                              reviews ratings         days  \
0                                    항상 맛있게 먹고 있습니다.       5  2024.03.26.   
1                너무 맛나요 ~ 짜지도 않고 적당해요~~ 다음에 또 구매할게용~       5  2024.03.25.   
2  지인추천으로 사먹고 맛있어서 두통재주문했어요. 집에서 고기먹을때 없으면 서운한 반찬...       5  2024.03.25.   
3  흠..저희 장아찌 좋아해요..근데 궁채에 고구마에서 나오는 길다린 씽?이라고해야하나...       4  2024.03.25.   

                                       options    buyer_categorys     유통기한  \
0                    유통기한꽤 남았어요포장꼼꼼해요맛 만족도맛있어요                      꽤 남았어요   
1        구매자거주인원 2인싱글유통기한꽤 남았어요포장꼼꼼해요맛 만족도맛있어요       구매자거주인원 2인싱글   꽤 남았어요   
2                   유통기한아주 넉넉해요포장꼼꼼해요맛 만족도맛있어요                     아주 넉넉해요   
3  구매자거주인원 5인취학자녀 가정유통기한아주 넉넉해요포장꼼꼼해요맛 만족도맛있어요  구매자거주인원 5인취학자녀 가정  아주 넉넉해요   

     포장 맛 만족도  
0  꼼꼼해요  맛있어요  
1  꼼꼼해요  맛있어요  
2  꼼꼼해요  맛있어요  
3  꼼꼼해요  맛있어요  
